# Photosensitization of CP1 by Xan, Delayed fluorescence

This is a simulation of CP1 sensitization by Xanthone according to differential equations below. $k_0$ is observed decay of Xanthone without any quencher.  $k_1 = k_q[\mathrm{CP1}]$ is the product of quenching rate constant and CP1 concentration. $k_d$ is a decay rate constant of diradical DR1 that is formed by photosensitization. The first two differential equations are selfexplanatory. The third describes the formation of singlet state of the diradical. This is formed mainly by T-T annihilation of $^3\mathrm{Xan}^*$ and $^3\mathrm{DR1}$, with the $k_{TT\ DA}$ (donor + acceptor) rate constant. Also, it could be formed by T-T with itself ($k_{TT\ AA}$, acceptor + acceptor), but this was not observed experimentally, but you can move the slider of $k_{TT\ AA}$ and see how the delayed fluorescence curve would look like if this process would be actually energetially feasible. The decay of $^1\mathrm{DR1}^*$ produces triplet $^3\mathrm{DR1}^*$ that emits light as a delayed fluorescence.


\begin{align}
\label{rce:vytezek}
\frac{\mathrm d[^3\mathrm{Xan}^*]}{\mathrm dt} &= - \left( k_0 + k_1\right) [^3\mathrm{Xan}^*]\\
\frac{\mathrm d[^3\mathrm{DR1}]}{\mathrm dt} &= k_1[^3\mathrm{Xan}^*] - k_d[^3\mathrm{DR1}]\\
\frac{\mathrm d[^1\mathrm{DR1}^*]}{\mathrm dt} &= k_{TT\ DA}[^3\mathrm{Xan}^*][^3\mathrm{DR1}] + k_{TT\ AA}[^3\mathrm{DR1}]^2 - k_{delay}[^1\mathrm{DR1}^*]\\
\end{align}

Because the measured delayed fluorescence curves are actually "shifted" to the right, there is a $k_{delay}$ rate constant which would cause that shift. This can be for example intersystem crossing from excited singlet to excited triplet state $^3\mathrm{DR1}^*$. If there is no delay ($k_{delay}[^1\mathrm{DR1}^*] >> k_{TT\ DA}[^3\mathrm{Xan}^*][^3\mathrm{DR1}]$), the delayed fluorescence intensity will be proporitonal only to product of $[^3\mathrm{Xan}^*]$ and $[^3\mathrm{DR1}]$ (check box No_Delay):

$$I_{df}\propto [^3\mathrm{Xan}^*][^3\mathrm{DR1}]$$


The correction for selfabosprtion of Xan is made by multiplying the obtained delayed fluorescence curve by derived light attenuation function $$T_{\mathrm{All}}(A) = \frac{1}{A\ln 10}\left(1-10^{-A}\right)$$ with the argument of product of Xan concentration and parameter $\mathbf{A_0}$, which is Xanthone concentration at time 0 - it was cca 0.2 from my measurements:

$$I_{df}\propto [^1\mathrm{DR1}^*]\times T_{\mathrm{All}}\left(A_0 [^3\mathrm{Xan}^*]\right)$$

The intensity is then normalized to value 0.3 so that the graph resembles plots in my report. You can see, that the self-absorption correction is indeed mild and for values of $A_0$ from 0 to 0.2 have almost no effect on the shift of the delayed fluorescence maximum to the right, only for values > 1, which was however not experimentally measured.

The varying of $k_{delay}$ actually shifts the delayed fluorescence maximum to the right, however, I am really not sure about this. If that was the case, maybe I should also observe that $^1\mathrm{DR1}^*$ in the transient absorption spectra (even though the concentration would be really low). I did not observe any other specie. Also, eg. setting the low delay rate constant that accurately shifts the maximum for low CP1 concentrations (low $k1$) produces unrealistic long decay tails at higher CP1 concentrations. These tails were not observed, acually, the delayed fluorescence curve at the highest measured CP1 concentration almost resembles the calculated curve (product of 630 nm and 390 nm curve - in my monthly report).

So in my opinion, the both additional processes ($k_{delay}$) a selfabsorption cannot explain the shifting of the maximum to the right. Some other explanation must hold there. Maybe the formation of exciplex between of $^3\mathrm{DR1}$ and ground state Xanthone, but I will try to model it after I read something in the literature.


** Varying TT parameter (either $k_{TT\ DA}$ or $k_{TT\ AA}$)  when the other is zero will have no effect on the shape of delayed fluorescence curve, because the curve is normalized. For calculations, I used numerical integration of differenctial equations. There is nice funtion odeint in Python, so it is actually really easy.

In [56]:
import matplotlib.pyplot as plt  # for plotting
import numpy as np  # numpy = Numerical Python
import ipywidgets as widgets
from scipy.integrate import odeint  # for numerical integration of differential equations

%matplotlib inline

In [107]:
times = np.linspace(0,2,500)

# for selfabsorption correction
def Tall(A):
    result = (1 - np.power(10, -A)) / (A * np.log(10))
    result[np.isnan(result)] = 1
    return result

@widgets.interact(k0=(0, 2, 0.2), k1=(0.1, 40, 0.5), kd=(0.5, 10, 1),
                  kTT_DA=(0.1, 1, 0.1), kTT_AA=(0, 1, 0.1), k_delay=(1, 100, 2), No_Delay=True, A0=(0, 2, 0.05))
def plot_func(k0, k1, kd, kTT_DA, kTT_AA, k_delay, No_Delay, A0):
  
    def solve(conc, t):
        cA, cB, cC = conc

        dcA_dt = -(k0 + k1) * cA
        dcB_dt = + k1 * cA - kd * cB
        dcC_dt = + kTT_DA * cA * cB + kTT_AA * cB ** 2 - k_delay * cC

        return [dcA_dt, dcB_dt, dcC_dt]
    
    result = odeint(solve, [1, 0, 0], times)
    
    Xan = result[:, 0]
    DR1 = result[:, 1]
    C = result[:, 2] if not No_Delay else kTT_DA * Xan * DR1 + kTT_AA * DR1 ** 2 # delayed fluorescence
    
    C *= Tall(Xan * A0) if A0 > 0 else 1  # selfabsorption correction
    
    C = C / np.max(C) * 0.3  # normalize delayed fluorescence curve to value 0.3
    
    fig, ax = plt.subplots()
    ax.set_xlim(-0.02, 2)
    ax.set_ylim(-0.02, 1.02)
    ax.set_ylabel('$c/c_0$')
    ax.set_xlabel('Time')
    
    plt.rcParams['figure.figsize'] = [15, 8]
    
    ax.plot(times, Xan , color='r', label='$^3Xan$')
    ax.plot(times, DR1 , color='b', label='$^3DR1$')
    ax.plot(times, C , color='black', ls=':', lw=3, label='Delayed fluorescence-norm.')
    
    ax.legend()

interactive(children=(FloatSlider(value=1.0, description='k0', max=2.0, step=0.2), FloatSlider(value=19.6, des…

In [105]:
times = np.linspace(0,2,1000)


@widgets.interact(k0=(0, 2, 0.2), kd=(0.5, 10, 1),
                  kTT_DA=(0.1, 1, 0.1), kTT_AA=(0, 1, 0.1), k_delay=(1, 100, 2), No_Delay=True, A0=(0, 2, 0.05))
def plot_func(k0, kd, kTT_DA, kTT_AA, k_delay, No_Delay, A0):
    def max_time(k1):
        def solve(conc, t):
            cA, cB, cC = conc

            dcA_dt = -(k0 + k1) * cA
            dcB_dt = + k1 * cA - kd * cB
            dcC_dt = + kTT_DA * cA * cB + kTT_AA * cB ** 2 - k_delay * cC

            return [dcA_dt, dcB_dt, dcC_dt]

        result = odeint(solve, [1, 0, 0], times)

        Xan = result[:, 0]
        DR1 = result[:, 1]
        C = result[:, 2] if not No_Delay else kTT_DA * Xan * DR1 + kTT_AA * DR1 ** 2

        C *= Tall(Xan * A0) if A0 > 0 else 1  # selfabsorption correction

        max_idx = np.argmax(C)
        max_time = times[max_idx]
        
        return max_time
    
    k1s = np.linspace(0.001, 100, num=1000)
    
    max_times = np.asarray([max_time(x) for x in k1s])
    
    plt.plot(k1s, max_times)
        
#     fig, ax = plt.subplots()
#     ax.set_xlim(-0.02, 2)
#     ax.set_ylim(-0.02, 1.02)
#     ax.set_ylabel('$c/c_0$')
#     ax.set_xlabel('Time')
    
#     plt.rcParams['figure.figsize'] = [15, 8]
    
#     ax.plot(times, Xan , color='r', label='$^3Xan$')
#     ax.plot(times, DR1 , color='b', label='$^3DR1$')
#     ax.plot(times, C , color='black', ls=':', lw=3, label='Delayed fluorescence-norm.')
    
#     ax.legend()

interactive(children=(FloatSlider(value=1.0, description='k0', max=2.0, step=0.2), FloatSlider(value=4.5, desc…

In [95]:
a = np.asarray([1, 1, 2, 5, 7, 8, 9, 6, 5, 2, 1 ,0])
np.argmax(a)

6